In [0]:
from pyspark.sql.functions import *

weatherSinkLayerName = "bronze"
weatherSinkStorageAccountName = "adlsudadalakehousedev"
weatherSinkFolderName = "weather-data"

weatherSinkFolderPath = f"abfss://{weatherSinkLayerName}@{weatherSinkStorageAccountName}.dfs.core.windows.net/{weatherSinkFolderName}"

weatherDF = ( spark 
             .read
             .json(weatherSinkFolderPath))

In [0]:
display(weatherDF)


In [0]:

output = weatherDF.select(
        col("daily.rain_sum").alias("rain_sum"),
        col("daily.temperature_2m_min").alias("temperature_2m_min"),
        col("daily.temperature_2m_max").alias("temperature_2m_max"),
        col("daily.time").alias("time"),
        col("daily_units.rain_sum").alias("unitOfRainFall"),
        col("daily_units.temperature_2m_max").alias("unitOfTemperature"),
        col("elevation"),
        col("generationtime_ms"),
        col("latitude"),
        col("longitude"),
        col("marketName"),
        col("timezone"),
        col("timezone_abbreviation"),
        col("utc_offset_seconds")
    )
display(output)



In [0]:
weatherDataDF= (output.withColumn(
     "zipped",
      arrays_zip("rain_sum", "temperature_2m_max","temperature_2m_min","time"))
     .withColumn("z",explode("zipped"))
     .select(
        col("z.rain_sum").alias("rainFall"),
        col("z.temperature_2m_max").alias("maximumTemparature"),
        col("z.temperature_2m_min").alias("minimumTemparature"),
        col("z.time").alias("weatherDate"),
        col("unitOfRainFall"),
        col("unitOfTemperature"),
        col("latitude"),
        col("longitude"),
        col("marketName")
     ).dropna(subset=["marketName","weatherDate"])
      .dropDuplicates(["marketName","weatherDate"])
        
        )
    

In [0]:
(weatherDataDF.withColumnRenamed('time', 'time_renamed')
 .write
 .mode("overwrite")
 .saveAsTable("pricing_analytics.silver.weather_data_silver"))

In [0]:
%sql
select count(*) from pricing_analytics.silver.weather_data_silver